In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import scipy.signal

In [29]:
class GameOfLife:
    def __init__(self, shape=(3, 3)):
        self.shape = shape
        self.state = np.zeros(self.shape, dtype=int)
        self.kernel = np.ones((3, 3), dtype=int)
        self.kernel[1, 1] = 0

    def load_grid_from_file(self, input_file):
        with open(input_file, "r") as f:
            width, height = map(int, f.readline().split())
            self.shape = (height, width)
            self.state = np.zeros(self.shape, dtype=bool)

            for line in f:
                value = line.split()
                if len(value) >= 2:
                    y, x = map(int, value[:2])
                    if 0 <= y < height and 0 <= x < width:
                        self.state[y, x] = 1
                else:
                    print("Line doesn't contain two values, check for floating lines at the bottom of the file")

    def update_state(self):
        conv_state = scipy.signal.convolve2d(self.state, self.kernel, mode="same")
        temp_state = np.zeros(shape=self.shape)
        temp_state[(conv_state < 2) | (conv_state > 3)] = 0
        temp_state[((conv_state == 2) | (conv_state == 3)) & (self.state == 1)] = 1
        temp_state[conv_state == 3] = 1
        self.state = temp_state

    def plot_state(self):
        return plt.imshow(self.state)

    def _animate(self, _):
        self.update_state()
        self.ax.clear()
        self.ax.imshow(self.state)
        self.ax.set(xticklabels=[], yticklabels=[])
        self.ax.tick_params(bottom=False, left=False)

    def animate(self, filename):
        self.fig, self.ax = plt.subplots(figsize=(12, 12), dpi=self.shape[0])
        ani = animation.FuncAnimation(self.fig, self._animate, frames=20, interval=10)
        ani.save(filename)
        return ani
    def save_state_to_file(self, output_file):
        with open(output_name, "w") as f:
            f.write(f"{game.shape[1]} {game.shape[0]}\n")
            for y in range(game.shape[0]):
                for x in range(game.shape[1]):
                    if game.state[y, x] == 1:
                        f.write(f"{y} {x}\n")

def run_game_no_dask(input_name, output_name, generations):
    game = GameOfLife()
    game.load_grid_from_file(input_name)

    for _ in range(generations):
        game.update_state()
    game.save_state_to_file(output_name)

In [30]:
if __name__ == "__main__":
    input_name = "./Data/1000x1000_0.1.txt"
    output_name = "Output_Files/output_1000x1000.txt"
    generations = 3

    game = GameOfLife()
    game.load_grid_from_file(input_name)
    game.save_state_to_file(output_name)

    Create an animation and save it to a GIF file
    game.animate("output_1000x1000_0.1.gif")

    plt.show()